In [1]:
from ete3 import NCBITaxa, Tree, TreeStyle
ncbi =NCBITaxa()

def get_name(tax_id):
    ncbi =NCBITaxa()
    name=str(ncbi.translate_to_names([tax_id]))[3:-2]
    return (name)
get_name(1071380)        

'Tetrapisispora blattae CBS 6284'

In [5]:
import xml.dom.minidom as minidom
 
def getTitles(xml):   
    doc = minidom.parse(xml)
    node = doc.documentElement
    taxonomies = doc.getElementsByTagName("taxonomy")
 
    for taxonomy in taxonomies:
        sci_name = taxonomy.getElementsByTagName("scientific_name")[0]
        nodes = sci_name.childNodes
        for node in nodes:
            if node.nodeType == node.TEXT_NODE:
                print node.data
 
if __name__ == "__main__":
    document = 'NCBIPhylo.xml'
    getTitles(document)


2759
33154
4751
451864
4890
716545
147538
716546
715989
147548
5178
28983
33196
40559
332648
5179
5180
665079
5181
101851
101852
1116229
76657
324777
698440
698441
1072389
159639
150173
149040
221903
34379
78156
655981
658429
342668
147550
222543
1028384
681950
5455
31870
645133
710243
1445577
80884
759273
474922
1213859
1033978
1036719
27337
498257
1051613
526221
5125
110618
5506
569360
5518
229533
101028
1028729
171631
5507
59765
426428
171627
117187
334819
232080
140110
70791
660122
474943
5581
176275
655819
45234
73501
983644
72232
114497
1081104
34397
243023
280754
1380566
5529
1650735
1276143
500148
1276141
92637
655827
568076
655844
474942
1052105
33203
5129
5543
63577
452589
29875
413071
51453
431241
398673
5592
5593
41687
563466
222544
639021
81093
148303
318829
242507
29849
36779
644352
5139
42302
5144
5145
515849
35718
1920207
78579
573729
5149
209285
285224
759272
38033
306901
35719
35720
578455
5148
5140
5141
367110
40127
510951
5146
5147
771870
1775898
1756146
65412
22319

# Get node Stats

In [6]:
from ete3 import NCBITaxa, Tree, TreeStyle
import numpy as np
import pandas as pd

write_header=True
write_assembler_header=True

ncbi =NCBITaxa()
df=pd.read_csv("stats.csv", names=['taxid','CDS','CDS_Mean','exon','exon_Mean','gene','gene_Mean','mRNA','mRNA_Mean'])
taxid_list=df['taxid']

def get_leaves_taxid(nodeset):
    ncbi =NCBITaxa()
    df=pd.read_csv("stats.csv", names=['taxid','CDS','CDS_Mean','exon','exon_Mean','gene','gene_Mean','mRNA','mRNA_Mean'])
    taxid_list=df['taxid']
    gff_set=set()
    
    for nodeid in nodeset:
        for taxid in taxid_list:
            if nodeid in ncbi.get_lineage(taxid):
                gff_set.add(taxid)
    return gff_set

def get_node_stats(tax_id): 
    
    info=df.loc[df['taxid'] == int(tax_id)].values.tolist()
    row=info[0]
    CDS=row[1]
    exon=row[3]
    gene=row[5]
    mRNA=row[7]
    tax2name= str(ncbi.translate_to_names([int(tax_id)]))[3:-2]
    return CDS,exon, gene, mRNA, tax2name

def get_node_describes(tax_id):
    taxid_list=get_leaves_taxid(set([int(tax_id)]))
    cds=df[df['taxid'].isin(taxid_list)]
    # another method to insert column to the datafram cds['parentTaxid']=tax_id
    cds.insert(0, "parentTaxid", tax_id)

    #write data fram to csv file
    # in an append mode
    global write_header
    if write_header==True:
        cds.to_csv('data1.csv', sep=',', mode='a', header=True)
        write_header=False
    else:
        cds.to_csv('data1.csv', sep=',', mode='a', header=False)
        
def get_node_assemblers(tax_id):
    taxid_list=get_leaves_taxid(set([int(tax_id)]))
    df=pd.read_csv("assemblers.csv", names=['taxid','assembler'])
    df=df[df['taxid'].isin(taxid_list)]
    df.insert(0, "parentTaxid", tax_id)
    #write data fram to csv file
    # in an append mode
    global write_assembler_header
    if write_assembler_header==True:
        df.to_csv('nodeAssembler.csv', sep=',', mode='a', header=True)
        write_assembler_header=False
    else:
        df.to_csv('nodeAssembler.csv', sep=',', mode='a', header=False)

        
def get_node_stats2(tax_id):
    taxid_list=get_leaves_taxid(set([int(tax_id)]))
    avg_CDS= df[df['taxid'].isin(taxid_list)]['CDS_Mean'].mean()
    avg_exon= df[df['taxid'].isin(taxid_list)]['exon_Mean'].mean()
    avg_gene= df[df['taxid'].isin(taxid_list)]['gene_Mean'].mean()
    avg_mRNA= df[df['taxid'].isin(taxid_list)]['mRNA_Mean'].mean()
    CDS_no= df[df['taxid'].isin(taxid_list)]['CDS'].mean()
    exon_no= df[df['taxid'].isin(taxid_list)]['exon'].mean()
    gene_no= df[df['taxid'].isin(taxid_list)]['gene'].mean()
    mRNA_no= df[df['taxid'].isin(taxid_list)]['mRNA'].mean()
    tax2name= str(ncbi.translate_to_names([int(tax_id)]))[3:-2]
    
    return avg_CDS,avg_exon, avg_gene, avg_mRNA,CDS_no,exon_no, gene_no, mRNA_no, tax2name


        
#get_node_assemblers(715989)
# get_node_describes(302)

# Annotate clade with new property

In [9]:
import xml.dom.minidom as minidom
 
def getTitles(xml):   
    doc = minidom.parse(xml)
    node = doc.documentElement
    clades = doc.getElementsByTagName("clade")
    
    taxonomy_list=[]
    for clade in clades:
        taxid=""
        taxonomy = clade.getElementsByTagName("taxonomy")[0]
        taxonomy_list.append(taxonomy)
        sci_name = taxonomy.getElementsByTagName("scientific_name")[0]
        nodes = sci_name.childNodes
        for node in nodes:
            if node.nodeType == node.TEXT_NODE:
                taxid= node.data

        # <property ref="avg_CDS" datatype="xsd:string" applies_to="node">avg_CDS</property>        
        avg_CDS,avg_exon, avg_gene, avg_mRNA,CDS_no,exon_no, gene_no, mRNA_no, tax2name= get_node_stats2(taxid)
        
        # write all the nodes stats in a csv file
        get_node_describes(taxid)
        
        # write all the assemblers programs in a csv file
        get_node_assemblers(taxid)

        
 
        x = doc.createElement("name")  
        txt = doc.createTextNode(tax2name)  
        x.appendChild(txt)  
        clade.appendChild(x)  

        x = doc.createElement("property")  
        x.setAttribute("ref", "avg_CDS")
        x.setAttribute("datatype", "xsd:string")
        x.setAttribute("applies_to", "node")
        txt = doc.createTextNode(str(avg_CDS))  
        x.appendChild(txt)  
        clade.appendChild(x)  
        
        x = doc.createElement("property")  
        x.setAttribute("ref", "avg_exon")
        x.setAttribute("datatype", "xsd:string")
        x.setAttribute("applies_to", "node")
        txt = doc.createTextNode(str(avg_exon))  
        x.appendChild(txt)  
        clade.appendChild(x)
        
        x = doc.createElement("property")  
        x.setAttribute("ref", "avg_gene")
        x.setAttribute("datatype", "xsd:string")
        x.setAttribute("applies_to", "node")
        txt = doc.createTextNode(str(avg_gene))  
        x.appendChild(txt)  
        clade.appendChild(x) 
    
        x = doc.createElement("property")  
        x.setAttribute("ref", "avg_mRNA")
        x.setAttribute("datatype", "xsd:string")
        x.setAttribute("applies_to", "node")
        txt = doc.createTextNode(str(avg_mRNA))  
        x.appendChild(txt)  
        clade.appendChild(x)  
        
        x = doc.createElement("property")  
        x.setAttribute("ref", "mRNA_no")
        x.setAttribute("datatype", "xsd:string")
        x.setAttribute("applies_to", "node")
        txt = doc.createTextNode(str(mRNA_no))  
        x.appendChild(txt)  
        clade.appendChild(x)  
        
        x = doc.createElement("property")  
        x.setAttribute("ref", "gene_no")
        x.setAttribute("datatype", "xsd:string")
        x.setAttribute("applies_to", "node")
        txt = doc.createTextNode(str(gene_no))  
        x.appendChild(txt)  
        clade.appendChild(x)  
        
        x = doc.createElement("property")  
        x.setAttribute("ref", "exon_no")
        x.setAttribute("datatype", "xsd:string")
        x.setAttribute("applies_to", "node")
        txt = doc.createTextNode(str(exon_no))  
        x.appendChild(txt)  
        clade.appendChild(x)  
        
        x = doc.createElement("property")  
        x.setAttribute("ref", "CDS_no")
        x.setAttribute("datatype", "xsd:string")
        x.setAttribute("applies_to", "node")
        txt = doc.createTextNode(str(CDS_no))  
        x.appendChild(txt)  
        clade.appendChild(x)  
        
        
        #<property ref="See also1" applies_to="clade" datatype="xsd:anyURI"> url </property>
        ncbi_url="https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=" + str(taxid)
        x = doc.createElement("property")  
        x.setAttribute("ref", "see ncbi:")
        x.setAttribute("datatype", "xsd:anyURI")
        x.setAttribute("applies_to", "node")
        txt = doc.createTextNode(ncbi_url)  
        x.appendChild(txt)  
        clade.appendChild(x)
        
        
    print(doc.toxml())
    with open ("NCBIPhylo_minidom.xml", 'w') as f:
        doc.writexml(f)

if __name__ == "__main__":
    document = 'tree2.xml'
    getTitles(document)

<?xml version="1.0" ?><phyloxml xmlns="http://www.phyloxml.org" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.phyloxml.org http://www.phyloxml.org/1.20/phyloxml.xsd">
<phylogeny rerootable="true" rooted="false">
  <clade>
    <clade>
      <branch_length>1.0</branch_length>
      <taxonomy>
        <scientific_name>2759</scientific_name>
      </taxonomy>
      <clade>
        <branch_length>1.0</branch_length>
        <taxonomy>
          <scientific_name>33154</scientific_name>
        </taxonomy>
        <clade>
          <branch_length>1.0</branch_length>
          <taxonomy>
            <scientific_name>4751</scientific_name>
          </taxonomy>
          <clade>
            <branch_length>1.0</branch_length>
            <taxonomy>
              <scientific_name>451864</scientific_name>
            </taxonomy>
            <clade>
              <branch_length>1.0</branch_length>
              <taxonomy>
                <scientific_name>4890<